# Rubin (DP0) & Roman (Troxel+23) images

Contact: Melissa Graham, Andrés A. Plazas Malagón

**Based on a tutorial by Chien-Hao Lin.**

Date: Mon Nov 11 2024

RSP Image: Weekly 2024_42

Goal: Recreate Chien-Hao's notebook and use Roman images to help with deblending Rubin images.

## Introduction

Space-based images have much higher resolution.
Stars and galaxies that are very close together or even overlapping (blended) due to chance
alignments along the line-of-sight can be better distinguished in higher resolution images.
So can actual galaxy mergers in close physical proximity, though this is less common.

It is possible to use the locations of physically distinct objects (deblended objects)
from higher resolution images to make more accurate photometry measurements.
In cases where the higher and lower resolution images are obtained in the same filters,
and with similar depths, it makes sense just to use the higher resolution images alone.

However, Rubin will obtain data in optical filters and Roman in infrared filters.
In this case, using the higher-resolution infrared images to determine the number and
location of distinct objects, and then make photometric measurements in Rubin's
optical-range images, can improve the optical photometric measurements.

Roman DC2 Simulated Images and Catalogs at IRSA IPAC:<br>
https://irsa.ipac.caltech.edu/data/theory/Roman/Troxel2023/overview.html

Troxel et al. (2023):<br>
https://academic.oup.com/mnras/article/522/2/2801/7076879?login=false

## Set up

In [59]:
import numpy as np

import lsst.geom
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
from lsst.daf.butler import Butler as dafButler

import lsst.daf.base as dafBase
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.detection import SourceDetectionTask
from lsst.meas.deblender import SourceDeblendTask
from lsst.meas.base import SingleFrameMeasurementTask
import lsst.afw.table as afwTable

import lsst.meas.algorithms as measAlg
import lsst.afw.math as afwMath
import lsst.afw.image as afwImage

Set the display backed to be Firefly.

In [2]:
afwDisplay.setDefaultBackend('firefly')

Instantiate the butler.

In [3]:
butler = dafButler('dp02', collections='2.2i/runs/DP0.2')

Define `ra` and `dec`, the central coordinates of interest.

Define the scale, in arcseconds per pixel, of Rubin and Roman images.

Define the stamp size to use when visualizing the images (i.e., the cutout size), in Rubin pixels; then use `stampsize / scale_ratio` as the extent when visualizing Roman images.

In [4]:
ra, dec = 54.28, -38.30
rubin_scale = 0.2
roman_scale = 0.0575
stampsize = 150
scale_ratio = rubin_scale/roman_scale

Convert the coordinates to type `SpherePoint`.

In [5]:
radec = lsst.geom.SpherePoint(ra, dec, lsst.geom.degrees)

For Rubin images: identify Rubin DP0.2 butler tract and patch for the desired coordinates.

In [6]:
skymap = butler.get('skyMap')
tract = skymap.findTract(radec).tract_id
patch = skymap.findTract(radec).findPatch(radec).getSequentialIndex()
print(tract, patch)

3633 9


For Roman images: four filters for one patch of deeply coadded Roman images have been stored in the shared space in the `/project` directory.

In [7]:
ro_img_path = '/project/melissagraham2/troxel2023/'
ro_img_fnms = ['dc2_Y106_54.24_-38.3.fits',
               'dc2_J129_54.24_-38.3.fits',
               'dc2_H158_54.24_-38.3.fits',
               'dc2_F184_54.24_-38.3.fits']

## Visualize images

### Rubin images

For each filter, retrieve the `deepCoadd` patch and display the cutout in Firefly frames 1 through 6.

In [8]:
lsst_bands = ['u', 'g', 'r', 'i', 'z', 'y']
for i in range(6):
    print(i+1, lsst_bands[i])
    afw_display = afwDisplay.Display(frame=i+1)
    dataId = {'tract': tract, 'patch': patch, 'band': lsst_bands[i]}
    image = butler.get('deepCoadd', dataId=dataId)
    size = stampsize
    extent = lsst.geom.ExtentI(size, size)
    cutout = image.getCutout(radec, extent)
    afw_display.mtv(cutout)
    afw_display.setMaskTransparency(100)
    del dataId, image, size, extent, cutout

1 u
2 g
3 r
4 i
5 z
6 y


### Roman images

Load and display a small cutout from each of the four images in Firefly frames 7 through 10.

> **Warnings:** Below, the warnings about unreadable mask extensions can be disregarded for the purposes of this tutorial, but generally when using the LSST Science Pipelines with non-Rubin data, all warnings should be followed up and third-party data sets might need to be reformatted to work properly.
In this case the images have four extensions: SCI, WHT, CTX, ERR.
But the `readFits` function expects MASK and IMAGE extensions.

In [9]:
for i in range(4):
    afw_display = afwDisplay.Display(frame=i+7)
    fnm = ro_img_path + ro_img_fnms[i]
    print(i+7, fnm)
    image = afwImage.ExposureF.readFits(fnm)
    size = stampsize * scale_ratio
    extent = lsst.geom.ExtentI(size, size)
    cutout = image.getCutout(radec, extent)
    afw_display.mtv(cutout)
    del fnm, image, size, extent, cutout

7 /project/melissagraham2/troxel2023/dc2_Y106_54.24_-38.3.fits


8 /project/melissagraham2/troxel2023/dc2_J129_54.24_-38.3.fits


9 /project/melissagraham2/troxel2023/dc2_H158_54.24_-38.3.fits


10 /project/melissagraham2/troxel2023/dc2_F184_54.24_-38.3.fits


### Clean up the Firefly window

Close each panel by clicking on the 'X' in the upper right corner until the display again says "Firefly Ready" and "Awaiting Python API Commands".

## Visualize detected sources

### Rubin

Source detection has already been run on the Rubin DP0 images.

The threshold is 5-sigma.

Display the r-band image cutout and overplot detected sources.

In [10]:
dataId = {'tract': tract, 'patch': patch, 'band': 'r'}

In [11]:
objects = butler.get('objectTable', dataId=dataId)

In [12]:
objects

column,shape_xy,footprintArea,detect_isIsolated,parentObjectId,coord_dec,yErr,shape_xx,refFwhm,y,shape_yy,...,r_kronFlux_flag,r_kronFlux_flag_bad_radius,r_kronFlux_flag_bad_shape,r_kronFlux_flag_bad_shape_no_psf,r_kronFlux_flag_edge,r_kronFlux_flag_no_fallback_radius,r_kronFlux_flag_no_minimum_radius,r_kronFlux_flag_small_radius,r_kronFlux_flag_used_minimum_radius,r_kronFlux_flag_used_psf_radius
objectId,,,,,,,,,,,,,,,,,,,,,
1565933256371601409,1.243153,303,False,0,-38.493715,NaN,4.673852,0.689904,3908.000000,6.097430,...,True,False,True,False,True,False,False,False,False,False
1565933256371601410,0.108741,324,True,0,-38.493890,NaN,3.654526,0.689725,3906.000000,4.147783,...,True,False,True,False,True,False,False,False,False,False
1565933256371601411,1.263088,623,False,0,-38.493916,NaN,4.018945,0.695862,3907.000000,6.841806,...,True,False,True,False,True,False,False,False,False,False
1565933256371601412,1.038074,1092,False,0,-38.494027,NaN,12.427411,0.696046,3906.000000,7.290579,...,True,False,True,False,True,False,False,False,False,False
1565933256371601413,NaN,160,True,0,-38.493986,NaN,NaN,0.695326,3907.000000,NaN,...,True,False,True,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1565933256371654383,-0.018556,132,True,1565933256371616347,-38.353039,0.985465,3.700394,0.782821,6450.848213,3.003776,...,False,True,False,False,False,False,False,False,False,True
1565933256371654384,0.088261,121,True,1565933256371616348,-38.349533,18.935143,3.532197,0.785004,6515.750069,3.047626,...,False,False,False,False,False,False,False,False,False,False
1565933256371654385,-0.113798,132,True,1565933256371616349,-38.345570,0.819093,3.919595,0.803746,6575.109889,3.743876,...,True,False,True,False,False,False,False,False,False,False


Extract coordinate data into `numpy` arrays.

In [13]:
ra_vals = np.asarray(objects['coord_ra'], dtype='float')
dec_vals = np.asarray(objects['coord_dec'], dtype='float')
x_vals = np.asarray(objects['x'], dtype='float')
y_vals = np.asarray(objects['y'], dtype='float')

Display the r-band cutout.

In [14]:
afw_display = afwDisplay.Display(frame=1)
image = butler.get('deepCoadd', dataId=dataId)
size = stampsize
extent = lsst.geom.ExtentI(size, size)
cutout = image.getCutout(radec, extent)
afw_display.mtv(cutout)
afw_display.setMaskTransparency(100)

Mark detected objects with orange circles.

The stampsize of 150 x 150 pixels is 30 x 30 arcsec, or about 0.008 x 0.008 degrees.
Only plot objects thata re in the cutout.

In [15]:
tx = np.where((ra_vals > ra - 0.01) &
              (ra_vals < ra + 0.01) &
              (dec_vals > dec - 0.01) &
              (dec_vals < dec + 0.01))[0]
print(len(tx))

232


In [16]:
with afw_display.Buffering():
    for x in tx:
        afw_display.dot('o', x_vals[x], y_vals[x],
                        size=5, ctype='orange')

Clean up.

In [17]:
del ra_vals, dec_vals, x_vals, y_vals
del tx
del dataId, image, size, extent, cutout

### Roman


#### Use DP0.2 tutorial 05 as a guide
Start by setting up the configurations for each task.

In [85]:
config_characterize = CharacterizeImageTask.ConfigClass()
config_characterize.psfIterations = 2

config_detection = SourceDetectionTask.ConfigClass()
config_detection.thresholdValue = 5
config_detection.thresholdType = "stdev"

config_deblend = SourceDeblendTask.ConfigClass()
config_deblend.propagateAllPeaks = True
config_deblend.maskPlanes=[]

config_measure = SingleFrameMeasurementTask.ConfigClass() 

In [61]:
schema = afwTable.SourceTable.makeMinimalSchema()
raerr = schema.addField("coord_raErr", type="F")
decerr = schema.addField("coord_decErr", type="F")

algMetadata = dafBase.PropertyList()

Define the tasks.

In [62]:
characterizeTask = CharacterizeImageTask(config=config_characterize)
detectionTask = SourceDetectionTask(schema=schema, config=config_detection)
deblendTask = SourceDeblendTask(schema=schema, config=config_deblend)
measureTask = SingleFrameMeasurementTask(schema=schema,
                                         config=config_measure,
                                         algMetadata=algMetadata)

Get the Roman image, set the PSF, and make a cutout.

In [63]:
fnm = ro_img_path + ro_img_fnms[2]
print(fnm)

/project/melissagraham2/troxel2023/dc2_H158_54.24_-38.3.fits


In [64]:
image = afwImage.ExposureF.readFits(fnm)


# Set PSF in full image
fn_psf = "/project/plazas/troxel2023/psf/coadd/dc2_H158_54.24_-38.3_psf.fits"
psf = measAlg.KernelPsf(afwMath.FixedKernel(afwImage.ImageD(fn_psf)))

image.setPsf(psf)

size = stampsize * scale_ratio
extent = lsst.geom.ExtentI(size, size)
cutout = image.getCutout(radec, extent)

In [65]:
tab = afwTable.SourceTable.make(schema)

In [66]:
result = detectionTask.run(tab, cutout)

In [67]:
result.numPosPeaks

26

In [68]:
sources = result.sources

In [69]:
# sources

In [70]:
deblendTask.run(cutout, sources)

In [71]:
measureTask.run(measCat=sources, exposure=cutout)

In [72]:
sources = sources.copy(True)

Display the Roman H-band cutout in frame 2.

In [74]:
afw_display = afwDisplay.Display(frame=2)
afw_display.mtv(cutout)
afw_display.setMaskTransparency(100)

Overplot detected sources.


In [75]:
ra_vals = np.asarray(sources['coord_ra'], dtype='float')
dec_vals = np.asarray(sources['coord_dec'], dtype='float')
x_vals = np.asarray(sources.getX(), dtype='float')
y_vals = np.asarray(sources.getY(), dtype='float')

In [76]:
print(np.min(ra_vals), np.max(ra_vals))
print(np.min(dec_vals), np.max(dec_vals))
print(np.min(x_vals), np.max(x_vals))
print(np.min(y_vals), np.max(y_vals))

0.9472796012143034 0.9474526559500056
-0.6685297899537634 -0.6683946434095236
2316.0 2803.0
5397.0 5881.800331373262


In [77]:
extent

Extent2I(521, 521)

In [78]:
cutout.image.getBBox()

Box2I(corner=Point2I(2303, 5384), dimensions=Extent2I(521, 521))

In [79]:
#tx = np.where((x_vals > 2303) &
#              (x_vals < 5384) &
#              (y_vals > 2303) &
#              (y_vals < 5384))[0]
# tx = np.where((ra_vals > ra - 0.01) &
#               (ra_vals < ra + 0.01) &
#               (dec_vals > dec - 0.01) &
#               (dec_vals < dec + 0.01))[0]
#print(len(tx))

Plot the fources.

In [80]:
with afw_display.Buffering():
    #for x in tx:
    for i in np.arange(len(x_vals)):
        afw_display.dot('o', x_vals[i], y_vals[i],
                        size=20, ctype='orange')

Clean up.

In [81]:
del ra_vals, dec_vals, x_vals, y_vals
del fnm, image, size, cutout, extent
del tab, result, sources

In [86]:
del config_detection, config_deblend, config_measure
del algMetadata, schema
del characterizeTask, detectionTask, deblendTask, measureTask